# Feature Selection

# Capstone Project - Predicting Traffic Accident Severity¶
## Applied DSc Capstone by IBM/Coursera


This notebook contains all the steps and transformations I performed for the feature selection for the predicting of Traffic Accident Severity project. You can find the information on the raw data in the following kaggle page. The kaggle data sets contain an extended descriptions of different aspect of the accidents, thus I've selected the most relevant and useful data for my analysis.

# Importing Libraries

In [23]:
import pandas as pd
import numpy as np
import seaborn as sns

import matplotlib.pyplot as plt

# Importing Data

In [24]:
df_Col = pd.read_csv('Data-Collisions.csv', low_memory=False)

 - Firstly I will go through the features and drop the ones that are not significant for the analysis
 - To provide a better insight some feature names will be modified
 - As we have the geographical coordinates available to reference if possible a heat map will be created to show the hotspots with most traffic accidents
 - With regards to the date and time of the accidents some modification will be done to provide two seperate features rather than combined.   

In [25]:
pd.set_option('max_columns', None)
df_Col.head()

,SEVERITYCODE,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,LOCATION,EXCEPTRSNCODE,EXCEPTRSNDESC,SEVERITYCODE.1,SEVERITYDESC,COLLISIONTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,INCDATE,INCDTTM,JUNCTIONTYPE,SDOT_COLCODE,SDOT_COLDESC,INATTENTIONIND,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,2,-122.323148,47.703140,1,1307,1307,3502005,Matched,Intersection,37475.0,5TH AVE NE AND NE 103RD ST,,NaN,2,Injury Collision,Angles,2,0,0,2,2013/03/27 00:00:00+00,3/27/2013 2:54:00 PM,At Intersection (intersection related),11,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END ...",NaN,N,Overcast,Wet,Daylight,NaN,NaN,NaN,10,Entering at angle,0,0,N
1,1,-122.347294,47.647172,2,52200,52200,2607959,Matched,Block,NaN,AURORA BR BETWEEN RAYE ST AND BRIDGE WAY N,NaN,NaN,1,Property Damage Only Collision,Sideswipe,2,0,0,2,2006/12/20 00:00:00+00,12/20/2006 6:55:00 PM,Mid-Block (not related to intersection),16,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, LEFT SIDE ...",NaN,0,Raining,Wet,Dark - Street Lights On,NaN,6354039.0,NaN,11,From same direction - both going straight - bo...,0,0,N
2,1,-122.334540,47.607871,3,26700,26700,1482393,Matched,Block,NaN,4TH AVE BETWEEN SENECA ST AND UNIVERSITY ST,NaN,NaN,1,Property Damage Only Collision,Parked Car,4,0,0,3,2004/11/18 00:00:00+00,11/18/2004 10:20:00 AM,Mid-Block (not related to intersection),14,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, REAR END",NaN,0,Overcast,Dry,Daylight,NaN,4323031.0,NaN,32,One parked--one moving,0,0,N
3,1,-122.334803,47.604803,4,1144,1144,3503937,Matched,Block,NaN,2ND AVE BETWEEN MARION ST AND MADISON ST,,NaN,1,Property Damage Only Collision,Other,3,0,0,3,2013/03/29 00:00:00+00,3/29/2013 9:26:00 AM,Mid-Block (not related to intersection),11,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END ...",NaN,N,Clear,Dry,Daylight,NaN,NaN,NaN,23,From same direction - all others,0,0,N
4,2,-122.306426,47.545739,5,17700,17700,1807429,Matched,Intersection,34387.0,SWIFT AVE S AND SWIFT AV OFF RP,NaN,NaN,2,Injury Collision,Angles,2,0,0,2,2004/01/28 00:00:00+00,1/28/2004 8:04:00 AM,At Intersection (intersection related),11,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END ...",NaN,0,Raining,Wet,Daylight,NaN,4028032.0,NaN,10,Entering at angle,0,0,N



The address type, location and junction type features relate to the location of the accidents. We also have the weather, road condition and light condition features that provide insight into the driving conditions at the time of the accident.
Using a combination of one or many of these columns we will gain a better insight into the risk factors associated with the accidents.

### Count Missing Values

In [26]:
# fixing data types
df_data_fix = df_Col.astype({'PEDROWNOTGRNT': 'bool'})
# calculating NaN count
df_data_fix.isna().sum().to_frame().rename(columns={0:'NaN Count'})

,NaN Count
SEVERITYCODE,0
X,5334
Y,5334
OBJECTID,0
INCKEY,0
COLDETKEY,0
REPORTNO,0
STATUS,0
ADDRTYPE,1926
INTKEY,129603


### Filling In Missing Values

In [27]:
print(df_Col.columns)

Index(['SEVERITYCODE', 'X', 'Y', 'OBJECTID', 'INCKEY', 'COLDETKEY', 'REPORTNO',
       'STATUS', 'ADDRTYPE', 'INTKEY', 'LOCATION', 'EXCEPTRSNCODE',
       'EXCEPTRSNDESC', 'SEVERITYCODE.1', 'SEVERITYDESC', 'COLLISIONTYPE',
       'PERSONCOUNT', 'PEDCOUNT', 'PEDCYLCOUNT', 'VEHCOUNT', 'INCDATE',
       'INCDTTM', 'JUNCTIONTYPE', 'SDOT_COLCODE', 'SDOT_COLDESC',
       'INATTENTIONIND', 'UNDERINFL', 'WEATHER', 'ROADCOND', 'LIGHTCOND',
       'PEDROWNOTGRNT', 'SDOTCOLNUM', 'SPEEDING', 'ST_COLCODE', 'ST_COLDESC',
       'SEGLANEKEY', 'CROSSWALKKEY', 'HITPARKEDCAR'],
      dtype='object')


In [28]:
df_accident = df_Col[["SEVERITYCODE","PERSONCOUNT","VEHCOUNT","WEATHER","ROADCOND","LIGHTCOND"]]
df_accident

,SEVERITYCODE,PERSONCOUNT,VEHCOUNT,WEATHER,ROADCOND,LIGHTCOND
0,2,2,2,Overcast,Wet,Daylight
1,1,2,2,Raining,Wet,Dark - Street Lights On
2,1,4,3,Overcast,Dry,Daylight
3,1,3,3,Clear,Dry,Daylight
4,2,2,2,Raining,Wet,Daylight
...,...,...,...,...,...,...
194668,2,3,2,Clear,Dry,Daylight
194669,1,2,2,Raining,Wet,Daylight
194670,2,3,2,Clear,Dry,Daylight
194671,2,2,1,Clear,Dry,Dusk


In [29]:
df_Light = df_Col["LIGHTCOND"]
df_Light.describe
df_Light.head(5)

0                   Daylight
1    Dark - Street Lights On
2                   Daylight
3                   Daylight
4                   Daylight
Name: LIGHTCOND, dtype: object

In [30]:
# Count the number of values in light condition
Light_Cat = df_Col['LIGHTCOND'].value_counts().sort_index()
Light_Cat

Dark - No Street Lights       1537
Dark - Street Lights Off      1199
Dark - Street Lights On      48507
Dark - Unknown Lighting         11
Dawn                          2502
Daylight                    116137
Dusk                          5902
Other                          235
Unknown                      13473
Name: LIGHTCOND, dtype: int64

Using the above data we are able to see that there is 7% of data within the LightCond column which is unknown, also when we take the 

| Light Conditions        | Count   | %  |
|:---                     |:----:   |:-- |
| Dark - No Street Lights |    1537 |(1%)|
| Dark - Street Lights Off|    1199 |(1%)|
| Dark - Street Lights On |    48507|(25%)|
| Dark - Unknown Lighting |      11 |(0%)|
| Dawn                    |     2502|(1%)|
| Daylight                |  116137 |(60%)|
| Dusk                    |     5902|(3%)|
| Other                   |      235|(0%)|
| Unknown                 |    13473|(7%)|

In [31]:
# we will try to fill out the missing rows in the light condition column with NA so that we can accurately see the factors as above currently we have 97-98% of data with classification.
df_Col["LIGHTCOND"].fillna("NA", inplace = True)
df_Col["LIGHTCOND"]

0                        Daylight
1         Dark - Street Lights On
2                        Daylight
3                        Daylight
4                        Daylight
                   ...           
194668                   Daylight
194669                   Daylight
194670                   Daylight
194671                       Dusk
194672                   Daylight
Name: LIGHTCOND, Length: 194673, dtype: object

In [32]:
# We run the same code as above to check whether the NA values that we assigned to the blank rows has been accepted.

Light_Cat = df_Col['LIGHTCOND'].value_counts().sort_index()
Light_Cat

Dark - No Street Lights       1537
Dark - Street Lights Off      1199
Dark - Street Lights On      48507
Dark - Unknown Lighting         11
Dawn                          2502
Daylight                    116137
Dusk                          5902
NA                            5170
Other                          235
Unknown                      13473
Name: LIGHTCOND, dtype: int64

Using the update table of data from above we can now fully understand how our data is split in this light condition column. As we can see below the NA rows make up 3% of the missing data having this filling in allows us to better judge the circumstances when looking at graphs later on.

| Light Conditions        | Count   | %  |
|:---                     |:----:   |:-- |
| Dark - No Street Lights |    1537 |(1%)|
| Dark - Street Lights Off|    1199 |(1%)|
| Dark - Street Lights On |    48507|(25%)|
| Dark - Unknown Lighting |      11 |(0%)|
| Dawn                    |     2502|(1%)|
| Daylight                |  116137 |(60%)|
| Dusk                    |     5902|(3%)|
| NA                      |     5170|(3%)|
| Other                   |      235|(0%)|
| Unknown                 |    13473|(7%)|

In [33]:
# Count the number of values in weather condition
Weather_Con = df_Col['WEATHER'].value_counts().sort_index()
Weather_Con

Blowing Sand/Dirt               56
Clear                       111135
Fog/Smog/Smoke                 569
Other                          832
Overcast                     27714
Partly Cloudy                    5
Raining                      33145
Severe Crosswind                25
Sleet/Hail/Freezing Rain       113
Snowing                        907
Unknown                      15091
Name: WEATHER, dtype: int64

Above with the weather data we can see how it is split with all the different factors, understanding these factors better and linking them with the light conditions and accident dates we will be able to get a better understanding of the conditions in which the risk factors are higher or lower.


| Weather Conditions      | Count  | Percentage  |
|:---                     |:----:  |:-- |
| Blowing Sand/Dirt       |    56  | 0% |
| Clear                   |  111135| 57%|
| Fog/Smog/Smoke          |   569  | 0% |
| Other                   |    832 | 0% |
| Overcast                |  27714 | 14%|
| Partly Cloudy           |  5     | 0% |
| Raining                 |   33145| 17%|
| Severe Crosswind        |    25  | 0% |
| Sleet/Hail/Freezing Rain|   113  | 0% |
| Snowing                 |   907  | 0% |
| Unknown                 | 15091  | 8% |

After looking at the above table and percentage distribution we can see that we have some data that is not classified, we should classify the data as 'NA' for the time being which will allow us to see how many instances have not been recorded.

In [34]:
# we will try to fill out the missing rows in the weather condition column with NA so that we can accurately see the factors as above currently we have 98% of data with classification.
df_Col["WEATHER"].fillna("NA", inplace = True)
df_Col["WEATHER"]

0         Overcast
1          Raining
2         Overcast
3            Clear
4          Raining
            ...   
194668       Clear
194669     Raining
194670       Clear
194671       Clear
194672       Clear
Name: WEATHER, Length: 194673, dtype: object

In [35]:
# We run the same code as above to check whether the NA values that we assigned to the blank rows has been accepted.

Weather_Con2 = df_Col['WEATHER'].value_counts().sort_index()
Weather_Con2

Blowing Sand/Dirt               56
Clear                       111135
Fog/Smog/Smoke                 569
NA                            5081
Other                          832
Overcast                     27714
Partly Cloudy                    5
Raining                      33145
Severe Crosswind                25
Sleet/Hail/Freezing Rain       113
Snowing                        907
Unknown                      15091
Name: WEATHER, dtype: int64

Looking at the above table we are able to see that we have a reasonable number of rows that have been given the label of 'NA', our table below will give us an idea of what this number represents interms of percentage.

| Weather Conditions      | Count  | Percentage  |
|:---                     |:----:  |:-- |
| Blowing Sand/Dirt       |    56  | 0% |
| Clear                   |  111135| 57%|
| Fog/Smog/Smoke          |   569  | 0% |
| NA                      | 5081   | 3% |
| Other                   |    832 | 0% |
| Overcast                |  27714 | 14%|
| Partly Cloudy           |  5     | 0% |
| Raining                 |   33145| 17%|
| Severe Crosswind        |    25  | 0% |
| Sleet/Hail/Freezing Rain|   113  | 0% |
| Snowing                 |   907  | 0% |
| Unknown                 | 15091  | 8% |

Now we can see that we have 3% of the weather data that has been given the classification of 'NA' with this in mind now we can rest assured that we have 100% of the data classified.

In [36]:
# Count the number of values in road condition
Road_Con = df_Col['ROADCOND'].value_counts().sort_index()
Road_Con

Dry               124510
Ice                 1209
Oil                   64
Other                132
Sand/Mud/Dirt         75
Snow/Slush          1004
Standing Water       115
Unknown            15078
Wet                47474
Name: ROADCOND, dtype: int64

Using the above data we are able to see the road conditions that have been reported at the time of accident, the below table shows us the percentage that each of the conditions make up.

| Road Conditions   | Count    | Percentage  |
|:---               |:----:    |:--          |
| Dry               |   124510 | 64% |
| Ice               |    1209  | 1% |
| Oil               |    64    | 0% |
| Other             |  132     | 0% |
| Sand/Mud/Dirt     |  75      | 0% |
| Snow/Slush        |  1004    | 1% |
| Standing Water    |  115     | 0% |
| Unknown           |  15078   | 8% |
| Wet               |  47474   | 24%|

Looking at the above table we can see that we have some missing data as the total of the percentage column comes to only 97%, inorder to get an idea of how many rows are not classified we will fill all the rows with no data with 'NA' this will give us an idea of the total number of rows affected.

In [37]:
# we will try to fill out the missing rows in the road condition column with NA so that we can accurately see the factors as above currently we have 97% of data with classification.
df_Col["ROADCOND"].fillna("NA", inplace = True)
df_Col["ROADCOND"]

0         Wet
1         Wet
2         Dry
3         Dry
4         Wet
         ... 
194668    Dry
194669    Wet
194670    Dry
194671    Dry
194672    Wet
Name: ROADCOND, Length: 194673, dtype: object

In [16]:
# We run the same code as above to check whether the NA values that we assigned to the blank rows has been accepted.

Road_Con2 = df_Col['ROADCOND'].value_counts().sort_index()
Road_Con2

Dry               124510
Ice                 1209
NA                  5012
Oil                   64
Other                132
Sand/Mud/Dirt         75
Snow/Slush          1004
Standing Water       115
Unknown            15078
Wet                47474
Name: ROADCOND, dtype: int64

Looking at the above table we can see that we have a total of 5012 rows that have been filled in with 'NA', with the help of the below table we can see that this makes up the remaining 3% that we were missing.

| Road Conditions   | Count    | Percentage  |
|:---               |:----:    |:--          |
| Dry               |   124510 | 64% |
| Ice               |    1209  | 1% |
| NA                |   5012   | 3% |
| Oil               |    64    | 0% |
| Other             |  132     | 0% |
| Sand/Mud/Dirt     |  75      | 0% |
| Snow/Slush        |  1004    | 1% |
| Standing Water    |  115     | 0% |
| Unknown           |  15078   | 8% |
| Wet               |  47474   | 24%|

In [38]:
# Count the number of values in vehicle count
Veh_Count = df_Col['VEHCOUNT'].value_counts().sort_index()
Veh_Count

0       5085
1      25748
2     147650
3      13010
4       2426
5        529
6        146
7         46
8         15
9          9
10         2
11         6
12         1
Name: VEHCOUNT, dtype: int64

With the above data we can see the number of vehicles invloved in an accident, with the help of the below table we can see that the highest percentage of vehicles invloved is 2.

| Vehicle Count  | Count    | Percentage  |
|:---            |:----:    |:--          |
| 0              |   5085   | 3% |
| 1              |    25748 | 13%|
| 2              |   147650 | 76%|
| 3              |    13010 | 7% |
| 4              |   2426   | 1% |
| 5              |  529     | 0% |
| 6              |  146     | 0% |
| 7              |  46      | 0% |
| 8              |  15      | 0% |
| 9              |  9       | 0% |
| 10             |  2       | 0% |
| 11             |  6       | 0% |
| 12             |  1       | 0% |

In [39]:
# Count the number of values in person count
Per_Count = df_Col['PERSONCOUNT'].value_counts().sort_index()
Per_Count

0       5544
1      13154
2     114231
3      35553
4      14660
5       6584
6       2702
7       1131
8        533
9        216
10       128
11        56
12        33
13        21
14        19
15        11
16         8
17        11
18         6
19         5
20         6
21         2
22         4
23         2
24         2
25         6
26         4
27         3
28         3
29         3
30         2
31         1
32         3
34         3
35         1
36         2
37         3
39         1
41         1
43         1
44         6
47         3
48         1
53         1
54         1
57         1
81         1
Name: PERSONCOUNT, dtype: int64

The above count shows us the number of people involved in the accidents, with the help of the table below we can see the largest percentage of people involved in accidents.

| Vehicle Count  | Count    | Percentage  |               | Vehicle Count  | Count    | Percentage  |       
|:---            |:----:    |:--          |               |   :---         | :----:   | :--         |
| 0              |    5544  | 3% |                        | 31             | 1        | 0%
| 1              |    13154 | 7% |                        | 32             | 3        | 0%
| 2              |   114231 | 59%|                        | 33             | 3        | 0%
| 3              |    35553 | 18%|                        | 34             | 3        | 0%
| 4              |    14660 | 8% |                        | 35             | 1        | 0%
| 5              |    6584  | 3% |                        | 36             | 2        | 0%
| 6              |    2702  | 1% |                        | 37             | 3        | 0%
| 7              |    1131  | 1% |                        | 38             | 1        | 0%
| 8              |    533   | 0% |                        | 39             | 1        | 0%
| 9              |    216   | 0% |                        | 41             | 1        | 0%
| 10             |    128   | 0% |                        | 43             | 1        | 0%
| 11             |    56    | 0% |                        | 44             | 6        | 0%
| 12             |    33    | 0% |                        | 47             | 3        | 0%
| 13             |    21    | 0% |                        | 48             | 1        | 0%
| 14             |    19    | 0% |                        | 53             | 1        | 0%
| 15             |    11    | 0% |                        | 54             | 1        | 0%
| 16             |    8     | 0% |                        | 57             | 1        | 0%
| 17             |    11    | 0% |                        | 81             | 1        | 0%
| 18             |    6     | 0% |                        
| 19             |    5     | 0% |                        
| 20             |    6     | 0% |                        
| 21             |    2     | 0% |                        
| 22             |    4     | 0% |                        
| 23             |    2     | 0% |                        
| 24             |    2     | 0% |                        
| 25             |    6     | 0% |                        
| 26             |    4     | 0% |                        
| 27             |    3     | 0% |                        
| 28             |    3     | 0% |                        
| 29             |    3     | 0% |                        
| 30             |    2     | 0% |                        

In [40]:
df_Col.shape

(194673, 38)

In [41]:
def missing_function(df_Col):
    missing_data = df_Col.isnull()
    missing_data.head()

    for column in missing_data.columns.values.tolist():
        print(column)
        print(missing_data[column].value_counts())
        print(df_Col[column].value_counts())
        print("")
        
missing_function(df_Col)

SEVERITYCODE
False    194673
Name: SEVERITYCODE, dtype: int64
1    136485
2     58188
Name: SEVERITYCODE, dtype: int64

X
False    189339
True       5334
Name: X, dtype: int64
-122.332653    265
-122.344896    254
-122.328079    252
-122.344997    239
-122.299160    231
              ... 
-122.322768      1
-122.288680      1
-122.405699      1
-122.323578      1
-122.343898      1
Name: X, Length: 23563, dtype: int64

Y
False    189339
True       5334
Name: Y, dtype: int64
47.708655    265
47.717173    254
47.604161    252
47.725036    239
47.579673    231
            ... 
47.556705      1
47.709101      1
47.513899      1
47.565438      1
47.563521      1
Name: Y, Length: 23839, dtype: int64

OBJECTID
False    194673
Name: OBJECTID, dtype: int64
2047     1
1194     1
58550    1
64693    1
62644    1
        ..
96890    1
90745    1
92792    1
70263    1
2049     1
Name: OBJECTID, Length: 194673, dtype: int64

INCKEY
False    194673
Name: INCKEY, dtype: int64
266238    1
81549     1
1

### Handling The Remaining Missing Values

In [43]:
df_Col.replace(r'^\s*$', np.nan, regex=True)
df_Col.replace("Unknown", np.nan, inplace = True)
df_Col.replace("Other", np.nan, inplace = True)
df_Col.replace("NA", np.nan, inplace = True)

print(df_Col.info())
missing_function(df_Col)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194673 entries, 0 to 194672
Data columns (total 38 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   SEVERITYCODE    194673 non-null  int64  
 1   X               189339 non-null  float64
 2   Y               189339 non-null  float64
 3   OBJECTID        194673 non-null  int64  
 4   INCKEY          194673 non-null  int64  
 5   COLDETKEY       194673 non-null  int64  
 6   REPORTNO        194673 non-null  object 
 7   STATUS          194673 non-null  object 
 8   ADDRTYPE        192747 non-null  object 
 9   INTKEY          65070 non-null   float64
 10  LOCATION        191996 non-null  object 
 11  EXCEPTRSNCODE   84811 non-null   object 
 12  EXCEPTRSNDESC   5638 non-null    object 
 13  SEVERITYCODE.1  194673 non-null  int64  
 14  SEVERITYDESC    194673 non-null  object 
 15  COLLISIONTYPE   166066 non-null  object 
 16  PERSONCOUNT     194673 non-null  int64  
 17  PEDCOUNT  

In [66]:
Sev_Count1 = df_Col["SEVERITYCODE"].value_counts().sort_index()
print('Total Severity Count of the Accidents:')
Sev_Count1

Total Severity Count of the Accidents:


1    136485
2     58188
Name: SEVERITYCODE, dtype: int64

Looking at the above data we can see an unbalance between the two severity codes classified as 1 & 2. As we can see that there is an unbalance in the data and that can cause a bias when running tests we will need to enure that the amount of data taken from the code 1 is the same as the code 2 total giving us a clean insight into the severity count.

In [67]:
df_Col.describe()

,SEVERITYCODE,X,Y,OBJECTID,INCKEY,COLDETKEY,INTKEY,SEVERITYCODE.1,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,SDOT_COLCODE,SDOTCOLNUM,SEGLANEKEY,CROSSWALKKEY
count,194673.000000,189339.000000,189339.000000,194673.000000,194673.000000,194673.000000,65070.000000,194673.000000,194673.000000,194673.000000,194673.000000,194673.000000,194673.000000,1.149360e+05,194673.000000,1.946730e+05
mean,1.298901,-122.330518,47.619543,108479.364930,141091.456350,141298.811381,37558.450576,1.298901,2.444427,0.037139,0.028391,1.920780,13.867768,7.972521e+06,269.401114,9.782452e+03
std,0.457778,0.029976,0.056157,62649.722558,86634.402737,86986.542110,51745.990273,0.457778,1.345929,0.198150,0.167413,0.631047,6.868755,2.553533e+06,3315.776055,7.226926e+04
min,1.000000,-122.419091,47.495573,1.000000,1001.000000,1001.000000,23807.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.007024e+06,0.000000,0.000000e+00
25%,1.000000,-122.348673,47.575956,54267.000000,70383.000000,70383.000000,28667.000000,1.000000,2.000000,0.000000,0.000000,2.000000,11.000000,6.040015e+06,0.000000,0.000000e+00
50%,1.000000,-122.330224,47.615369,106912.000000,123363.000000,123363.000000,29973.000000,1.000000,2.000000,0.000000,0.000000,2.000000,13.000000,8.023022e+06,0.000000,0.000000e+00
75%,2.000000,-122.311937,47.663664,162272.000000,203319.000000,203459.000000,33973.000000,2.000000,3.000000,0.000000,0.000000,2.000000,14.000000,1.015501e+07,0.000000,0.000000e+00
max,2.000000,-122.238949,47.734142,219547.000000,331454.000000,332954.000000,757580.000000,2.000000,81.000000,6.000000,2.000000,12.000000,69.000000,1.307202e+07,525241.000000,5.239700e+06


In [69]:
# dropping unrelevant columns 

df_Col.drop(['SEVERITYCODE.1'], axis=1, inplace=True)
df_Col.columns

Index(['SEVERITYCODE', 'X', 'Y', 'OBJECTID', 'INCKEY', 'COLDETKEY', 'REPORTNO',
       'STATUS', 'ADDRTYPE', 'INTKEY', 'LOCATION', 'EXCEPTRSNCODE',
       'EXCEPTRSNDESC', 'SEVERITYDESC', 'COLLISIONTYPE', 'PERSONCOUNT',
       'PEDCOUNT', 'PEDCYLCOUNT', 'VEHCOUNT', 'INCDATE', 'INCDTTM',
       'JUNCTIONTYPE', 'SDOT_COLCODE', 'SDOT_COLDESC', 'INATTENTIONIND',
       'UNDERINFL', 'WEATHER', 'ROADCOND', 'LIGHTCOND', 'PEDROWNOTGRNT',
       'SDOTCOLNUM', 'SPEEDING', 'ST_COLCODE', 'ST_COLDESC', 'SEGLANEKEY',
       'CROSSWALKKEY', 'HITPARKEDCAR'],
      dtype='object')

In [70]:
df_Col['SEVERITYCODE'].value_counts()

1    136485
2     58188
Name: SEVERITYCODE, dtype: int64

In [71]:
df_Col.to_csv('Cleansed_Collisions2.csv')